<a href="https://colab.research.google.com/github/nataliiakrasilynets/Sales-Analytics/blob/main/Portfolio_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Завантаження даних та бібліотек

In [ ]:
# Connecting Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Changing work folder
%cd /content/drive/MyDrive/PythonforDataAnalytics/

/content/drive/MyDrive/PythonforDataAnalytics


In [ ]:
#Завантажуємо бібліотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
ab = pd.read_csv("A_B_testing.csv")

In [ ]:
ab

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1
...,...,...,...,...,...,...,...,...,...
800991,2020-11-22,United Arab Emirates,mobile,Asia,Organic Search,1,2,session_start,1
800992,2020-12-16,United Arab Emirates,desktop,Asia,Undefined,4,1,scroll,1
800993,2020-12-17,United Arab Emirates,mobile,Asia,Direct,3,1,scroll,1
800994,2020-12-19,United Arab Emirates,desktop,Asia,Paid Search,4,1,page_view,1


#Підготовлення даних (а саме створення dataframe для групи 1 та групи 2 з відповідними критеріями)

In [ ]:
#Створення datafrema з значеннями групи 1 для add_payment_info,add_shipping_info,begin_checkout та new account
ab_where_event_name = ab[(ab["event_name"] == "add_payment_info") | (ab["event_name"] == "add_shipping_info") | (ab["event_name"] == "begin_checkout") | (ab["event_name"] == "new account")]
ab_where_event_name_group = ab_where_event_name.groupby(["test", "test_group", "event_name"])["value"].sum().reset_index()
ab_where_event_name_group_1=ab_where_event_name_group[ab_where_event_name_group["test_group"] == 1]
ab_where_event_name_group_1.insert(3, 'session', 'session')
ab_where_event_name_group_1.loc[:, 'metric'] = ab_where_event_name_group_1['event_name'].astype(str) + "/" + ab_where_event_name_group_1['session'].astype(str)
cols = ab_where_event_name_group_1.columns.tolist()
cols.insert(1, cols.pop(cols.index('metric')))
ab_where_event_name_group_1 = ab_where_event_name_group_1[cols]
#Створення datafrema з значеннями групи 2 для add_payment_info,add_shipping_info,begin_checkout та new account
ab_where_event_name_group_2=ab_where_event_name_group[ab_where_event_name_group["test_group"] == 2]

In [ ]:
#Підрахунок session за групою 1 та 2
ab_where_session = ab[(ab["event_name"] == "session")]
ab_where_session_group = ab_where_session.groupby(["test", "test_group"])["value"].sum().reset_index()
ab_where_session_group1=ab_where_session_group[ab_where_session_group["test_group"] == 1]
ab_where_session_group2=ab_where_session_group[ab_where_session_group["test_group"] == 2]

#Об'єднання підготовлених даних в один Dateframe

In [ ]:
#Об'єднання результатів event_name i session для кожної групи
ab_where_event_name_and_session_for_group_1 = ab_where_event_name_group_1.merge(ab_where_session_group1[['test', 'test_group', "value"]], on=['test', 'test_group'], how='inner')
ab_where_event_name_and_session_for_group_2 = ab_where_event_name_group_2.merge(ab_where_session_group2[['test', 'test_group', "value"]], on=['test', 'test_group'], how='inner')

In [ ]:
#Видалення непотрібних стовпців та змінення назв стовпців
total_for_group_1 = ab_where_event_name_and_session_for_group_1.drop(columns=["test_group"])
total_for_group_1 = total_for_group_1.rename(columns={'value_x': 'numerator_for_group_1', 'value_y': 'denominator_for_group_1'})
total_for_group_2 = ab_where_event_name_and_session_for_group_2.drop(columns=["test_group"])
total_for_group_2 = total_for_group_2.rename(columns={'value_x': 'numerator_for_group_2', 'value_y': 'denominator_for_group_2'})

In [ ]:
#Об'єднання результатів 2-ох груп
total = total_for_group_1.merge(total_for_group_2[['test', 'event_name', "numerator_for_group_2", "denominator_for_group_2"]], on=['test', 'event_name'], how='inner')

#Конверсія

In [ ]:
#Порахуємо конверсію
total.loc[:, 'conversion_for_group_1'] = total['numerator_for_group_1'] / total['denominator_for_group_1']
col = total.columns.tolist()
col.insert(6, col.pop(col.index('conversion_for_group_1')))
total = total[col]
total.loc[:, 'conversion_for_group_2'] = total['numerator_for_group_2'] / total['denominator_for_group_2']
total["metric_change"] = (total["conversion_for_group_2"] / total["conversion_for_group_1"] - 1)*100

#Перевірка на нормальність

In [ ]:
#перевірка на нормальність кожної групи, тесту, та за event_name
from scipy.stats import shapiro
import scipy.stats as stats
ab_where_1 = ab[(ab["event_name"] == "new account")]

def check_normality(values):
  stat, p = shapiro(values)
  return p > 0.05
for group in ab_where_1['test_group'].unique():
  for test in ab_where_1['test'].unique():
    subset = ab_where_1[(ab_where_1['test_group'] == group) & (ab_where_1['test'] == test)]['value']
    if len(subset) > 0:
      is_normal = check_normality(subset)
      result = "ок" if is_normal else "не ок"
      print(f'Група {group}, Тест {test}: Дані {result}')

Група 2, Тест 2: Дані не ок
Група 2, Тест 1: Дані не ок
Група 2, Тест 4: Дані не ок
Група 2, Тест 3: Дані не ок
Група 1, Тест 2: Дані не ок
Група 1, Тест 1: Дані не ок
Група 1, Тест 4: Дані не ок
Група 1, Тест 3: Дані не ок


In [ ]:
from statsmodels.stats.proportion import proportions_ztest
def calculate_z_stat_and_p_value(row):
    successes = [row["numerator_for_group_1"], row["numerator_for_group_2"]]
    totals = [row["denominator_for_group_1"], row["denominator_for_group_2"]]
    z_stat, p_value = proportions_ztest(successes, totals, alternative='two-sided')
    return pd.Series({"z_stat": z_stat, "p_value": p_value})
total[["z_stat", "p_value"]] = total.apply(calculate_z_stat_and_p_value, axis=1)
total["significant"] = total["p_value"] < 0.05

#Збереження даних до текстового файлу

In [ ]:
total.to_csv('output.tsv', sep='\t', index=False)

#Результати

1. [Dashboard1](https://public.tableau.com/app/profile/nataliia.krasilynets/viz/PortfolioProject1_17359277313610/Dashboard1?publish=yes)
[Dashboard2](https://public.tableau.com/app/profile/nataliia.krasilynets/viz/ABtest_17354958345990/ABtest?publish=yes)
2. [tsv](https://drive.google.com/file/d/1-2jVqKa9A7aiU-8bTTJgpyL_qEBZ24dV/view?usp=sharing)

# Висновки:
1. Зміни в тесті 1 позитивно повпливали на метрику:
- add_payment_info/session та збільшили її на 12.5%.
- add_shipping_info/session та збільшили її на 6.5%.
- begin_checkout/session та збільшили її на 6.6%.

З вірогідністю 95%, даний вплив не є випадковим.

Для метрики new account/session	вплив є випадковим, якщо, навіть, і збільшимо вибірку для статистичної значущості, то матимемо просідку.

2. Тест 2 жодна із метрик не дала позитивного результату. Для того щоб тест був статистично значущим потрібна вибірка 2*1,126,212=2252424
3. В тесті 3/4 лише одна/дві метрики дали позитивний результат, при  цьому маємо в них просідку
4. Тест 4 жодна із метрик не є статистично значущою.

###Тому лише для 3-ох метрик add_payment_info/session, add_shipping_info/session та begin_checkout/session можемо ввести зміни з тесту 1


In [ ]:
total

,test,metric,event_name,session,numerator_for_group_1,denominator_for_group_1,conversion_for_group_1,numerator_for_group_2,denominator_for_group_2,conversion_for_group_2,metric_change,z_stat,p_value,significant
0,1,add_payment_info/session,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,-3.924884,0.000087,True
1,1,add_shipping_info/session,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,-2.603571,0.009226,True
2,1,begin_checkout/session,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,-2.978783,0.002894,True
3,1,new account/session,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,1.542883,0.122859,False
4,2,add_payment_info/session,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,False
5,2,add_shipping_info/session,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,-0.709557,0.477979,False
6,2,begin_checkout/session,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,-0.952898,0.340642,False
7,2,new account/session,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,-0.588793,0.556000,False
8,3,add_payment_info/session,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,-0.643172,0.520112,False
9,3,add_shipping_info/session,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,1.413727,0.157442,False
